W całym niniejszym notebooku będziemy zakładać, że naszym modelem jest regresja logistyczna z parametrami $w,b$, a jej output oznacza prawdopodobieństwo klasy $y=1$.

## Accuracy - podejście klasyczne

Oznaczmy przez $m_{w,b}$ output naszego modelu. $m_{w,b}$ jest funkcją, która danej obserwacji $x\in X$ przyporządkowuje rozkład prawdopodobieństwa na etykietach $y\in Y$. Zadaniem tej funkcji jest modelować dla każdego $x$ prawdziwy rozkład warunkowy $p(y\mid x)$
$$\forall_{x\in X}: m_{w,b}(x) \approx p(y\mid x)$$
Jeśli $Y$ jest zbiorem skończonym, to funkcja $m_{w,b}$ zwraca tyle liczb, ile jest elementów $Y$. W wypadku, gdy $Y$ ma dwa elementy możemy uprościć model i stwierdzić, że funkcja $m_{w,b}$ zwraca jedynie prawdopodobieństwo drugiego elementu, a prawdopodobieństwo pierwszego wyliczamy korzystając z faktu, że suma prawdopodobieństw wynosi $1$:
$$\left\{\begin{array}{l}\hat p(y=1\mid x) := m_{w,b}(x) \\ \hat p(y=0\mid x) := 1 - m_{w,b}(x)\end{array}\right .$$

Od tej pory $m_{w,b}(x)$ będzie w zależności od kontekstu oznaczało rozkład na $Y$ lub liczbowe prawdopodobieństwo klasy $y=1$.

Uczenie polega na znalezieniu takich parametrów $w, b$, żeby powyższe przybliżenie było średnio (zgodnie z prawdopodobieństwem $p$) jak najlepsze. Uczenie gradientowe polega na wprowadzeniu różniczkowalnej funkcji kosztu, której argumentami są rozkłady $m_{w,b}(x)$ oraz $p(y\mid x)$, a wartość jest tym mniejsza, im podobniejsze są do siebie te dwa rozkłady (wartość zero oznacza równość rozkładów). Następnie wystarczy zróżniczkować tę funkcję po parametrach $w,b$ i możemy minimalizować gradientowo koszt.

Załóżmy, że chcemy maksymalizować średnie accuracy, tzn. liczbę poprawnych predykcji etykiety $y$ na podstawie zaobserwowanego $x$. Od teraz będziemy nazywać tę predykcję __akcją__ (action). W ogólnym przypadku wybieranie akcji po zaobserwowaniu konkretnego $x$ nie musi być deterministyczne (tzw. strategie mieszane, mixed strategies), dlatego zdefiniujmy funkcję $\pi$, która dla każdego $x$ zwraca rozkład prawdopodobieństwa na zbiorze $Y$ i rozkład ten oznacza tym razem, jak często wybieramy daną etykietę. Oczywiście $\pi(x) \neq p(y\mid x)$ - jeśli np. dla danego $x$ etykieta $y=1$ ma prawdopodobieństwo $60\%$, a etykieta $y=0$ tylko $40\%$, to nigdy nie opłaca się przewidywać $y=0$. Wzór na optymalne $\pi(x)$ jest następujący:

$$\pi(x) = \left\{\begin{array}{llll}
    (y=0) \mapsto 0; (y=1)\mapsto 1 & \mathrm{gdy} & p(y=1\mid x) > 0,5 \\
    (y=0) \mapsto 1; (y=1)\mapsto 0 & \mathrm{gdy} & p(y=1\mid x) < 0,5 \\
    (y=0) \mapsto q; (y=1)\mapsto 1-q & \mathrm{gdy} & p(y=1\mid x) = 0,5 & q\in[0,1]\\
\end{array}\right .$$

Czyli gdy $y=1$ ma prawdopodobieństwo większe niż $50\%$, to zawsze podejmujemy akcję $y=1$. Jeśli to prawdopodobieństwo jest mniejsze od $50\%$, to zawsze podejmujemy akcję $y=0$. Jeśli z kolei jest równe dokładnie $50\%$, to możemy przyjąć dowolną strategię [dlaczego?]. Dla ułatwienia można wziąć np. $q=0$ i wtedy wzór przyjmuje postać:

$$\pi(x) = \left\{\begin{array}{llll}
    (y=0) \mapsto 0; (y=1)\mapsto 1 & \mathrm{gdy} & p(y=1\mid x) \geq 0,5 \\
    (y=0) \mapsto 1; (y=1)\mapsto 0 & \mathrm{gdy} & p(y=1\mid x) < 0,5 \\
\end{array}\right .$$

Standardowe podejście, które stosowaliśmy dotychczas, wygląda w następujący sposób: najpierw trenujemy funkcję $m_{w,b}(x)$, która przybliża $p(y\mid x)$, a następnie definiujemy:

$$\pi_{w,b}(x) := \pi_{m_{w,b}}(x) := \left\{\begin{array}{llll}
    (y=0) \mapsto 0; (y=1)\mapsto 1 & \mathrm{gdy} & m_{w,b}(x) \geq 0,5 \\
    (y=0) \mapsto 1; (y=1)\mapsto 0 & \mathrm{gdy} & m_{w,b}(x) < 0,5 \\
\end{array}\right .$$

$m_{w,b}(x)$ było do tej pory implementowane metodą `predict_proba`, a $\pi_{m_{w,b}}(x)$ metodą `predict` (tu korzystaliśmy z faktu, że dla każdego $x$ predykcja jest faktycznie deterministyczna, więc nie zwracaliśmy rozkładu prawdopodobieństwa, tylko wartość tego $y$, który ma prawdopodobieństwo $100\%$).

Ale skoro interesuje nas tylko accuracy, to dlaczego nie uczymy modelu, który __od razu przewiduje akcję__?

Czyli zamiast:
$$ m_{w,b}(x) \sim p(y\mid x);\ \pi_{w,b} := \pi_{m_{w,b}} $$
uczyć od razu model:
$$ \pi_{w,b}(x) \sim \pi(y\mid x) $$



## "Nieróżniczkowalność" accuracy

Powszechnie mówi się, że accuracy jest nieróżniczkowalne - w takich sytuacjach chodzi o funkcję:
$$acc(y\_true, y\_pred) = \dfrac{1}{n}\sum_{j=1}^n (y\_true_j == y\_pred_j)$$

Nieporozumienie polega na tym, że $y\_pred$ jest rozumiane jako zestaw wylosowanych etykiet, a nie __prawdopodobieństwa__ $\pi_{w,b}(x)$, z jakimi te etykiety były losowane. Jeśli natomiast $y\_pred$ będzie oznaczało prawdopodobieństwa, to możemy napisać [dlaczego to wciąż jest accuracy?]:
$$acc(y\_true, y\_pred) = \dfrac{1}{n}\sum_{j=1}^n y\_true_j \cdot y\_pred_j + (1 - y\_true_j) \cdot (1 - y\_pred_j)$$
i ta funkcja jest już jak najbardziej różniczkowalna. Możemy więc zdefiniować loss jako:
$$L(y\_true, y\_pred) := - acc(y\_true, y\_pred) = -\dfrac{1}{n}\sum_{j=1}^n y\_true_j \cdot y\_pred_j + (1 - y\_true_j) \cdot (1 - y\_pred_j)$$

Wzór ten jest bardzo podobny do kosztu binary cross-entropy - trzeba dobrze zrozumieć, czym się różnią, żeby nigdy ich nie pomylić! Przede wszystkim zwróćmy uwagę na postać optymalnego `y_pred`, które minimalizuje te koszty. Jeśli np. dla pewnego ustalonego $x$ mamy $p(y=1\mid x) = 60\%$, to optymalizacja wzorem binary cross-entropy będzie dążyła do ustawienia predykcji dla tego $x$ na wartość $0,6$, podczas gdy optymalizacja accuracy będzie dążyła do wartośći $1$.

## Dlaczego nie możemy użyć wzoru na binary cross-entropy do optymalizacji accuracy?

W powyższej sekcji musieliśmy "ręcznie" wprowadzić nowy wzór na różniczkowalne accuracy. Dlaczego nie da się użyć znanych wzorów do uczenia parametrów funkcji $\pi_{w,b}$? Otóż żeby to zrobić, musielibyśmy wstawić do wzoru na binary cross-entropy funkcję $\pi_{w,b}$ jako `y_pred` oraz rozkład $\pi$ jako `y_true`. Problem polega na tym, że __nie mamy danych pochodzących z rozkładu $\pi$__, ponieważ nasz dataset __pochodzi z rozkładu $p$__.

Jak wyglądałby dataset pochodzący z rozkładu $\pi$? Otóż byłby to zestaw odpowiedzi "wyroczni" na pytanie "dla danego przykładu $x$, która z etykiet ze zbioru $Y$ jest najbardziej prawdopodobna?" - czyli jeśli dla pewnego $x$ zachodzi $p(y=1\mid x)=60\%$, to wyrocznia zawsze odpowie "$y=1$", pomimo że w naszym oryginalnym datasecie jest spora szansa ($40$ procent), że etykieta będzie równa $0$.

## Formalne wyprowadzenie wzoru na accuracy

Accuracy to średnia liczba odgadnięć poprawnej etykiety. Oznaczmy literą $a \in Y$ akcję, którą podejmujemy dla ustalonego $x\in X$. Zbiór akcji oczywiście jest równy zbiorowi etykiet. Pary $x, y$ pochodzą z rozkładu $p$, podczas gdy akcja $a$ losowana jest z rozkładu $\pi_{w,b}(x)$, gdzie $w,b$ to parametry, których się uczymy, a $x$ to obserwacja.

Wprowadźmy jeszcze pojęcie __nagrody__, którą oznaczymy literą $r \in \mathbb{R}$. W wypadku accuracy nagroda wynosi $1$, gdy udało nam się poprawnie odgadnąć etykietę, i $0$ w przeciwnym przypadku:

$$r_{acc}(y,a) := \left\{\begin{array}{lll}
    1 & \mathrm{gdy} & y = a \\
    0 & \mathrm{gdy} & y \neq a \\
\end{array}\right .$$

Oznaczmy funkcją $\upsilon(w,b)$ średnią nagrodę, którą uzyskamy stosując strategię $\pi_{w,b}$ (czyli tutaj $\upsilon(w,b)$ to po prostu accuracy modelu z parametrami $w, b$).

Spróbujmy wyprowadzić krok po kroku wzór na średnią nagrodę:

$$\upsilon(w,b) = \ldots$$

pierwszą średnią liczymy po wszystkich możliwych obserwacjach $x$ oraz prawdziwych etykietach $y$:

$$\upsilon(w,b) = \underset{x,y\sim p}{\mathbb{E}} \ldots$$

w tym momencie możemy zaobserwować jedynie $x$ i na tej podstawie wylosować akcję $a$, którą podejmiemy:

$$\upsilon(w,b) = \underset{x,y\sim p}{\mathbb{E}}\ \underset{a\sim\pi_{w,b}(x)}{\mathbb{E}}\ldots$$

na koniec musimy po prostu wstawić do wzoru wartość nagrody otrzymanej dla danych $x,y,a$:

$$\upsilon(w,b) = \underset{x,y\sim p}{\mathbb{E}}\ \underset{a\sim\pi_{w,b}(x)}{\mathbb{E}}r_{acc}(y,a)$$

Drugą wartość oczekiwaną możemy zamienić na całkę:

$$\upsilon(w,b) = \underset{x,y\sim p}{\mathbb{E}}\ \underset{Y}{\int}\pi_{w,b}(x)(a)\cdot r_{acc}(y,a)\ da$$

($\pi_{w,b}(x)(a)$ oznacza prawdopodobieństwo akcji $a$ w rozkładzie $\pi_{w,b}(x)$)

zamieniając całki na sumy, podstawiając z definicji wartości $r_{acc}$ i grupując odpowiednio składniki [liczymy przy tablicy], otrzymujemy wzór na accuracy z poprzedniej sekcji:

$$\upsilon(w,b) = \underset{x,y\sim p}{\mathbb{E}}\ \underset{Y}{\int}\pi_{w,b}(x)(a)\cdot r_{acc}(y,a)\ da \sim \dfrac{1}{n}\sum_{j=1}^n y\_true_j \cdot y\_pred_j + (1 - y\_true_j) \cdot (1 - y\_pred_j)$$

#### Uwaga na zapis $\pi_{w,b}(x)(a)$

Jeśli $\pi_{w,b}(x)$ to output regresji logistycznej dla obserwacji $x$, to:
$$\begin{align}
\pi_{w,b}(x)(1) &= \pi_{w,b}(x) \\
\pi_{w,b}(x)(0) &= 1 - \pi_{w,b}(x)
\end{align}$$
i takie wartości należy wstawić do wzoru!

### Policy gradient z epizodem o długości 1

Wróćmy do wzoru:

$$\upsilon(w,b) = \underset{x,y\sim p}{\mathbb{E}}\ \underset{a\sim\pi_{w,b}(x)}{\mathbb{E}}r_{acc}(y,a)$$

i spróbujmy go zróżniczkować po parametrach $w, b$ - w ten sposób wyprowadzimy algorytm uczenia średniego accuracy:

$$\begin{align}
\dfrac{\partial\upsilon(w,b)}{\partial w} &= \dfrac{\partial}{\partial w}\underset{x,y\sim p}{\mathbb{E}}\ \underset{a\sim\pi_{w,b}(x)}{\mathbb{E}}r_{acc}(y,a) = \\
&=\underset{x,y\sim p}{\mathbb{E}}\ \dfrac{\partial}{\partial w}\underset{a\sim\pi_{w,b}(x)}{\mathbb{E}}r_{acc}(y,a) = \\
&=\underset{x,y\sim p}{\mathbb{E}}\ \dfrac{\partial}{\partial w} \underset{Y}{\int}\pi_{w,b}(x)(a)\cdot r_{acc}(y,a)\ da = \\
&=\underset{x,y\sim p}{\mathbb{E}}\ \underset{Y}{\int}\dfrac{\partial \pi_{w,b}(x)(a)}{\partial w}\cdot r_{acc}(y,a)\ da = \ldots
\end{align}$$

Umiemy policzyć $\dfrac{\partial \pi_{w,b}(x)(a)}{\partial w}$, bo znamy wzór na nasz model. Wykonajmy jeszcze jeden trick, który sprawi, że całka zamieni się z powrotem na wartość oczekiwaną [dlaczego nie chcemy tu całki? wyjaśni się pod koniec tej sekcji]:

$$\begin{align}
\ldots&=\underset{x,y\sim p}{\mathbb{E}}\ \underset{Y}{\int}\dfrac{\pi_{w,b}(x)(a)}{\pi_{w,b}(x)(a)} \dfrac{\partial \pi_{w,b}(x)(a)}{\partial w}\cdot r_{acc}(y,a)\ da = \\
&=\underset{x,y\sim p}{\mathbb{E}}\ \underset{a\sim\pi_{w,b}(x)}{\mathbb{E}}\dfrac{1}{\pi_{w,b}(x)(a)} \dfrac{\partial \pi_{w,b}(x)(a)}{\partial w}\cdot r_{acc}(y,a) = \ldots
\end{align}$$

Ostatnia modyfikacja - wyłącznie w celu skrócenia zapisu:

$$\ldots = \underset{x,y\sim p}{\mathbb{E}}\ \underset{a\sim\pi_{w,b}(x)}{\mathbb{E}}\dfrac{\partial(\ln\pi_{w,b}(x)(a))}{\partial w}\cdot r_{acc}(y,a)$$

Ostatecznie:
$$\dfrac{\partial\upsilon(w,b)}{\partial w} = \underset{x,y\sim p}{\mathbb{E}}\ \underset{a\sim\pi_{w,b}(x)}{\mathbb{E}}\dfrac{\partial(\ln\pi_{w,b}(x)(a))}{\partial w}\cdot r_{acc}(y,a)$$

i analogicznie:
$$\dfrac{\partial\upsilon(w,b)}{\partial b} = \underset{x,y\sim p}{\mathbb{E}}\ \underset{a\sim\pi_{w,b}(x)}{\mathbb{E}}\dfrac{\partial(\ln\pi_{w,b}(x)(a))}{\partial b}\cdot r_{acc}(y,a)$$

W związku z tym pochodną $\dfrac{\partial\upsilon(w,b)}{\partial w}$ (analogicznie pochodna po $b$) można aproksymować w następujący sposób:
1. Wylosuj dane $x,y$ z prawdziwego rozkładu (lub weź ze zbioru treningowego).
2. Wylosuj akcję $a$ z rozkładu $\pi_{w,b}(x)$.
3. Policz pochodną $\dfrac{\partial(\ln\pi_{w,b}(x)(a))}{\partial w}$ i przemnóż ją przez nagrodę $r_{acc}(y,a)$.
4. Powtórz kroki 1-3 wielokrotnie, wyniki uśrednij.

#### Uogólnienie na dowolną nagrodę

Zauważmy, że wyprowadzając wzór na gradient średniej nagrody nie musieliśmy rozumieć, jak działa $r_{acc}$ - wystarczy jedynie znać jej wartość. Okazuje się, że $r$ może być praktycznie dowolną funkcją - nie musi być deterministyczna, różniczkowalna, a nawet ciągła!

Policy gradient mówi o tym, jak poprawić parametry naszej strategii, żeby zwiększyć średnią nagrodę. Wprowadza się dwa pojęcia: agent i środowisko. Strategia jest częścią agenta, który wykonuje akcje i uczy się maksymalizować średnią nagrodę, natomiast środowisko dostarcza obserwacji i liczbowych wartości nagrody.

W tym ogólnym przypadku wzory na gradient wyglądają następująco:
$$\begin{align}
\dfrac{\partial\upsilon(w,b)}{\partial w} &= \underset{x\sim p}{\mathbb{E}}\ \underset{a\sim\pi_{w,b}(x)}{\mathbb{E}}\dfrac{\partial(\ln\pi_{w,b}(x)(a))}{\partial w}\cdot r(a) \\
\dfrac{\partial\upsilon(w,b)}{\partial b} &= \underset{x\sim p}{\mathbb{E}}\ \underset{a\sim\pi_{w,b}(x)}{\mathbb{E}}\dfrac{\partial(\ln\pi_{w,b}(x)(a))}{\partial b}\cdot r(a)
\end{align}$$

a uczenie agenta przebiega w pętli:
1. Środowisko losuje obserwację $x$ z nieznanego rozkładu $p$ i przekazuje ją agentowi.
2. Agent losuje akcję $a$ z rozkładu $\pi_{w,b}(x)$ i przekazuje ją do środowiska.
3. Środowisko zwraca agentowi nagrodę $r(a)$.
4. Agent oblicza pochodną $\dfrac{\partial(\ln\pi_{w,b}(x)(a))}{\partial w}$ i mnoży ją przez nagrodę $r(a)$.
5. Agent powtarza kroki 1-4 wielokrotnie, a następnie uśrednia wyniki z kroku 4, mnoży je przez learning rate i dokonuje pojedynczego update'u swoich parametrów.
6. Kroki 1-5 powtarzamy do czasu, aż agent się nauczy.

Proszę pamiętać, że w kroku 5 __nie dopisujemy znaku minus przy gradiencie__ - znak minus pojawia się przy __minimalizowaniu__ funkcji kosztu, natomiast tutaj chcemy __maksymalizować__ średnią nagrodę.

#### Co oznacza "epizod o długości 1"?

Nagroda wypłacana przez środowisko może zależeć nie tylko od akcji, którą wykonaliśmy w danym kroku, ale też od całej historii interakcji agenta ze środowiskiem. Długość epizodu to liczba kolejnych wypłacanych nagród, które są od siebie zależne.

W powyższych wzorach zakładaliśmy, że wszystkie akcje są niezależne - losowaliśmy obserwację $x$ z ustalonego (ale nieznanego) rozkładu $p$, wykonywaliśmy akcję $a$, a nagroda była obliczana tylko na podstawie tych dwóch wielkości. Kolejna iteracja pętli przebiegała całkowicie niezależnie od poprzedniej.

Można rozumieć to tak, że epizod o długości 1 oznacza, iż obserwacje są I.I.D., a środowisko nie ma żadnej "pamięci", w której mogłoby przechowywać akcje wykonane przez agenta (lub środowisko jest bezstanowe, czyli agent nie może go zmodyfikować). To założenie sprawdza się w problemie klasyfikacji, ale w praktyce oczywiście taka sytuacja nie występuje nigdy - na następnych ćwiczeniach wprowadzimy agenta, który uczy się z dłuższych epizodów.

#### Odpowiedź na pytanie

Gdyby we wzorze na policy gradient zamiast drugiej wartości oczekiwanej wstawić całkę, to zamiast samplować jedną akcję $a$ musielibyśmy policzyć sumę po wszystkich możliwych akcjach. W wypadku maksymalizowania accuracy da się to oczywiście zrobić (i tak będzie lepiej, bo zamiast aproksymacji mamy wartość dokładną), ale w ogólnym przypadku środowisko nie pozwoli nam wykonać więcej niż jednej akcji dla danej obserwacji $x$, czyli nie mamy możliwości poznania wartości reward dla wszystkich akcji. 